In [3]:
import json
import os
import re
import pandas as pd
from shutil import copy, move

pokedex_path = '../Version20/Pokedex/'
abilities_path = '../Version20/Abilities/'
moves_path = '../Version20/Moves/'
learnsets_path = '../Version20/Learnsets/'
natures_path = '../Version20/Natures/'
items_path = '../Version20/Items/'
paths = [pokedex_path,abilities_path,moves_path,learnsets_path,natures_path,items_path]

# Image Cleanup

In [3]:
updates = pd.read_csv('../Sources/img_updates.csv')
for i, row in updates.iterrows():
    # print(row)
    if type(row['source']) != str:
        continue #skips empty sources
    if row['type'] == "book":
        outpath = '../images/BookSprites/'
        srcpath = '../Sources/Dreamworld Color/'
    elif row['type'] == "home":
        outpath = '../images/HomeSprites/'
        srcpath = '../Sources/Pokémon HOME/'
    elif row['type'] == "box":
        outpath = '../images/BookSprites/'
        srcpath = '../Sources/pokesprite/pokemon-gen8/regular/'
    else:
        print(row)
        break
    
    # print(srcpath+row['source'], outpath+row['target'])
    assert os.path.exists(srcpath+row['source']), f"{row['source']} doesn't exist"
    # assert os.path.exists(outpath+row['target']), f"{row['target']} doesn't exist"
    try:
        copy(srcpath+row['source'], outpath+row['target'])
    except:
        print(row)
        raise

# Cell Operation:

- Adds MegaBaseForm to Pokedex
- slugify's the _id field of all data points

In [7]:

# for fname in os.listdir(pokedex_path):
#     if '.json' not in fname: continue
#     path = pokedex_path+fname
#     data = json.loads(open(path).read())
#     if 'Mega-' in data['Name']:
#         basename = re.search("Mega-(\w+)",data['Name']).group(1)
#         data['MegaBaseForm'] = basename
#     open(path,'w').write(json.dumps(data, indent=4))

# for data_path in paths:
#     for fname in os.listdir(data_path):
#         if '.json' not in fname: continue
#         path = data_path+fname
#         data = json.loads(open(path).read())
        
#         data['_id'] = data['_id'].lower().replace(' ', '-')
#         open(path,'w').write(json.dumps(data, indent=4))


# Merge moves.json Metadata

In [ ]:
new = json.loads(open('../Sources/moves.json').read())
for move in new:
    name = move['Name']
    path = moves_path+name+'.json'
    data = json.loads(open(path).read())
    data['Attributes'] = move['Attributes']
    data['AddedEffects'] = move['AddedEffects']
    print(move, data)
    open(path,'w').write(json.dumps(data, indent=4))

# Evolution Tree Parsing

In [69]:
def _entry(entry):
    # try:
        f = entry['From']['@DisplayName']
        t = entry['To']['@DisplayName']
        k = entry['Kind']
        d = json.loads(entry['Details']) if entry['Details'] else ""
        d = {k.capitalize():v for k,v in d.items()} if type(d) == dict else ""
        
        x = { "From": f, "To": t, "Kind": k}
        x.update(d)
        return x
    # except: 
    #     print(entry)
    #     raise Exception

data = json.loads(open('../raw/XmlDump/EvolutionTrees.json').read())
compiled = []
for tree in data:
    entries = tree['EvolutionEntries']['EvolutionEntry']
    if type(entries) == dict:
        compiled.append(_entry(entries))
    else:
        for entry in entries:
            compiled.append(_entry(entry))

for fname in os.listdir(pokedex_path):
    if '.json' not in fname: continue
    path = pokedex_path+fname
    data = json.loads(open(path).read())
    name = data['Name']
    data['Evolutions'] = []
    try:
        for entry in compiled:
            if entry['From'] == name:
                data['Evolutions'].append(entry.copy())
                del data['Evolutions'][-1]['From']
                if data['Evolutions'][-1]["Kind"] == "Mega" and "Mega" not in name:
                    for e in data['MegaEvolutions']:
                        if e['Name'] == data['Evolutions'][-1]['To']:
                            # data['Evolutions'][-1]['Details'] = {"item": e['Item']}
                            data['Evolutions'][-1]['Item'] = e['Item']
                            break
            if entry["To"] == name:
                data['Evolutions'].append(entry.copy())
                if data['Evolutions'][-1]["Kind"] == "Mega" and "Mega" not in name:
                    for e in data['MegaEvolutions']:
                        if e['Name'] == data['Evolutions'][-1]['To']:
                            # data['Evolutions'][-1]['Details'] = {"item": e['Item']}
                            data['Evolutions'][-1]['Item'] = e['Item']
                            break
                if data['Evolutions'][-1]["Kind"] == "Mega" and "Mega" in name:
                    base = json.loads(open(pokedex_path+data['Evolutions'][-1]['From']+'.json').read())
                    for e in base['MegaEvolutions']:
                        if e['Name'] == data['Evolutions'][-1]['To']:
                            # data['Evolutions'][-1]['Details'] = {"item": e['Item']}
                            data['Evolutions'][-1]['Item'] = e['Item']
                            break
                del data['Evolutions'][-1]['To']
    except:
        print(data, entry)
        raise Exception
    open(path,'w').write(json.dumps(data, indent=4))

In [70]:
for fname in os.listdir(pokedex_path):
    if '.json' not in fname: continue
    path = pokedex_path+fname
    data = json.loads(open(path).read())
    data.pop('MegaEvolutions', None)
    data.pop('MegaBaseForm', None)
    open(path,'w').write(json.dumps(data, indent=4))